PARTE 1 - BAIXAR E TREINAR OS DADOS

In [70]:
# Importar a base de dados
import pandas as pd 

df = pd.read_csv('dados/ENEM/microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv', sep=';', encoding='latin1')

In [79]:
# FILTRA PARA TER DADOS PRECISOS
# TIRA OS TREINEIROS
# TIRA QUEM NÃO FEZ AS DUAS PROVAS (ELIMINADO OU FALTOU) - NÚMEROS 0 E 2 
# TIRA QUE NÃO TEM A REDAÇÃO CORRIGIDA

# TIRA TREINEIRO (1 - Treineiro, 0 - Não Treineiro)
filtra_treineiros = df["IN_TREINEIRO"] == 0

# PEGAR PRESENÇA - PRECISA SER 1
# CIÊNCIA DAS NATUREZAS
filtra_CN = df["TP_PRESENCA_CN"] == 1
# CIÊNCIA HUMANAS
filtra_CH = df["TP_PRESENCA_CH"] == 1
# LINGUAGENS E CÓDIGOS
filtra_ling = df["TP_PRESENCA_LC"] == 1
# MATEMÁTICA
filtra_mat = df["TP_PRESENCA_MT"] == 1

# REDAÇÕES NÃO CORRIGIDAS (MELHOR TIRAR O STATUS IGUAL A 2 - ANULADA)
filtra_red = df["TP_STATUS_REDACAO"] != 2

df = df[filtra_treineiros & filtra_CN & filtra_CH & filtra_ling & filtra_mat & filtra_red]



In [80]:

df_focado = pd.DataFrame()

# PEGA RENDA FAMILIAR
df_focado["RENDA"] = df["Q006"]

# PEGAR NOTAS

# Nota de redação
df_focado["NOTA REDAÇÃO"] = df["NU_NOTA_REDACAO"]
# Nota de Ciências da Natureza
df_focado["NOTA CIENCIA DA NATUREZA"] = df["NU_NOTA_CN"]
# Nota de Ciências Humanas
df_focado["NOTA CIENCIAS HUMANAS"] = df["NU_NOTA_CH"]
# Nota de Linguagens e Códigos
df_focado["NOTA LINGUAGEM E CODIGOS"] = df["NU_NOTA_LC"]
# Nota de Matemática
df_focado["NOTA MATEMATICA"] = df["NU_NOTA_MT"]

df_focado
# df_compara["MEDIA ALUNO"] = (df_compara["NOTA REDAÇÃO"] + df_compara["NOTA CIENCIA DA NATUREZA"] + df_compara["NOTA CIENCIAS HUMANAS"] + df_compara["NOTA LINGUAGEM E CODIGOS"] + df_compara["NOTA MATEMATICA"]) / 5


,RENDA,NOTA REDAÇÃO,NOTA CIENCIA DA NATUREZA,NOTA CIENCIAS HUMANAS,NOTA LINGUAGEM E CODIGOS,NOTA MATEMATICA
2,B,760.0,421.1,546.0,498.8,565.3
3,A,320.0,490.7,388.6,357.8,416.0
7,B,440.0,398.1,427.3,400.2,404.9
9,B,360.0,467.5,461.0,466.7,435.3
11,B,940.0,458.7,539.8,488.2,456.8
...,...,...,...,...,...,...
3476095,C,580.0,444.5,504.4,489.5,423.6
3476097,D,740.0,536.1,633.2,584.0,596.3
3476098,B,580.0,487.6,495.6,545.5,597.4
3476099,B,520.0,512.5,524.8,546.8,432.0


In [73]:
df_rendas_notas = pd.DataFrame()
renda_categorias = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q"]

for categoria_social in renda_categorias:

    # FILTRA PELA RENDA A
    df_renda_a = df_focado.loc[df_focado['RENDA'] == categoria_social]

    media_renda_a_redacao = df_renda_a["NOTA REDAÇÃO"].mean()
    media_renda_a_cnatureza = df_renda_a["NOTA CIENCIA DA NATUREZA"].mean()
    media_renda_a_chumanas = df_renda_a["NOTA CIENCIAS HUMANAS"].mean()
    media_renda_a_clinguagem = df_renda_a["NOTA LINGUAGEM E CODIGOS"].mean()
    media_renda_a_cmatematica = df_renda_a["NOTA MATEMATICA"].mean()

    df_rendas_notas["RENDA"] = "OI"
    print(f"CATEGORIA {categoria_social}")
    print(f"Média REDAÇÃO: {media_renda_a_redacao}")
    print(f"Média CIENCIA DA NATUREZA: {media_renda_a_cnatureza}")
    print(f"Média CIENCIAS HUMANAS: {media_renda_a_chumanas}")
    print(f"Média LINGUAGEM E CODIGOS: {media_renda_a_clinguagem}")
    print(f"Média MATEMATICA: {media_renda_a_cmatematica}")

CATEGORIA A
Média REDAÇÃO: 515.4452081361612
Média CIENCIA DA NATUREZA: 459.88145177427964
Média CIENCIAS HUMANAS: 481.14920329542446
Média LINGUAGEM E CODIGOS: 466.606901152778
Média MATEMATICA: 472.7826282350587
CATEGORIA B
Média REDAÇÃO: 553.5394984168764
Média CIENCIA DA NATUREZA: 468.8328754275104
Média CIENCIAS HUMANAS: 495.29728006680693
Média LINGUAGEM E CODIGOS: 485.22313470271814
Média MATEMATICA: 491.94201617512846
CATEGORIA C
Média REDAÇÃO: 590.9498203356767
Média CIENCIA DA NATUREZA: 483.0637125832418
Média CIENCIAS HUMANAS: 515.6012189883205
Média LINGUAGEM E CODIGOS: 508.6682615771039
Média MATEMATICA: 520.8451286460032
CATEGORIA D
Média REDAÇÃO: 612.5494951141725
Média CIENCIA DA NATUREZA: 491.59158890952807
Média CIENCIAS HUMANAS: 525.9692921408994
Média LINGUAGEM E CODIGOS: 519.3811077938884
Média MATEMATICA: 536.9990006767752
CATEGORIA E
Média REDAÇÃO: 635.1857161601976
Média CIENCIA DA NATUREZA: 500.2172741752924
Média CIENCIAS HUMANAS: 535.5395177162962
Média LINGU